In [8]:
from datetime import datetime, timedelta, timezone
from typing import List, Dict, Union
import os
import dotenv
import requests
from dateutil import parser
import pandas as pd
import vectorbt as vbt
import numpy as np
import time
import pprint

In [20]:
dotenv.load_dotenv()

def take_daily_tokens_movement(
  wallet_address:str,
  chain_name:str,
  days_lookback:int
) -> Dict[str, List[str]]: # {date: [coin's contract adress 1, coin's contract adress 2]}

  url = f"https://api.covalenthq.com/v1/{chain_name}/address/{wallet_address}/portfolio_v2/"

  query_params = {"days": days_lookback}

  headers = {"Authorization": f"Bearer {os.getenv('GOLDRUSH_API_KEY')}"}

  response = requests.request("GET", url, headers=headers, params=query_params)

  tokens_movement = {}

  for coin_data in response.json()["data"]["items"]:
    prev_item = None

    for item in coin_data["holdings"]:
      if prev_item is None:
        prev_item = item
        continue
      
      prev_day_close = float(prev_item["close"]["balance"]) 

      cur_day_high = float(item["high"]["balance"]) 
      cur_day_low = float(item["low"]["balance"]) 
      cur_day_close = float(item["close"]["balance"]) 

      if (
        cur_day_close != prev_day_close or
        cur_day_low != prev_day_close or
        cur_day_high != prev_day_close
      ):
        if tokens_movement.get(item["timestamp"], None) is None:
          tokens_movement[item["timestamp"]] = []
        
        tokens_movement[item["timestamp"]].append(coin_data["contract_address"])
      
      prev_item = item
    
  return tokens_movement

def get_a_block(chain_name:str, block_height: Union[str | int] = "latest"):
  while True:
    url = f"https://api.covalenthq.com/v1/{chain_name}/block_v2/{block_height}/"

    headers = {"Authorization": f"Bearer {os.getenv('GOLDRUSH_API_KEY')}"}

    response = requests.request("GET", url, headers=headers)

    if not response.ok:
      print(response.text)
      print("sleeping for 30 sec")
      time.sleep(30)
      continue

    return response.json()["data"]["items"][0]

def take_starting_block_height(chain_name, days_lookback:int):
  from_datetime = datetime.now(tz=timezone.utc) - timedelta(days=days_lookback)
  threshold_window = timedelta(minutes=15)

  latest_block_h = get_a_block(chain_name, "latest")["height"]
  low = 0
  high = latest_block_h

  while low <= high:
    mid = (low + high) // 2
    block = get_a_block(chain_name, mid)
    block_dt = parser.parse(block["signed_at"])

    if from_datetime - threshold_window <= block_dt <= from_datetime + threshold_window:
      return mid

    if block_dt < from_datetime:
      low = mid + 1
    else:
      high = mid - 1

  raise ValueError("Could not find block close enough to target date.")

def take_all_erc_token_transfers(wallet_address:str, chain_name:str, token_address:str, starting_block:int):
  url = f"https://api.covalenthq.com/v1/{chain_name}/address/{wallet_address}/transfers_v2/"

  
  headers = {"Authorization": f"Bearer {os.getenv('GOLDRUSH_API_KEY')}"}

  all_transfers = []

  p = 0
  while True:
    query_params = {"starting-block": starting_block, "page-number": p, "quote-currency": "EUR", "contract-address": token_address}
    response = requests.request("GET", url, headers=headers, params=query_params)
    response_data = response.json()["data"]
    
    if not response.ok:
      print(response.text)
      print("slepping for 30 sec")
      time.sleep(30)
      continue

    for item in response_data["items"]:
      all_transfers += item["transfers"]
    
    if response_data["pagination"]["has_more"]:
      p += 1
    else:
      break

  return all_transfers

def backtest_token_portfolio(token_address: str, wallet_address: str, token_transfers: list):
    prices = {}
    position_deltas = {}

    for transfer in token_transfers:
        dt = parser.parse(transfer["block_signed_at"])
        quote_rate = transfer.get("quote_rate", 0.0)

        if quote_rate is None or quote_rate <= 0:
            continue

        token_decimals = transfer.get("contract_decimals", 0.0)
        amount = float(transfer["delta"]) / (10 ** token_decimals)

        # Determine if wallet gained or lost tokens
        delta = -amount if transfer["from_address"] == wallet_address else amount

        prices[dt] = quote_rate
        position_deltas[dt] = delta  # now raw (not cumulative)

    if not prices:
        return None

    price_series = pd.Series(prices).sort_index()
    delta_series = pd.Series(position_deltas).sort_index()

    # Generate entry signals based on positive/negative delta
    long_entries = delta_series > 0
    short_entries = delta_series < 0

    # Use absolute delta as position size
    size_series = delta_series.abs()

    return backtest_with_vectorbt(price_series, long_entries, short_entries, size_series)



def backtest_with_vectorbt(price_series: pd.Series, long_entries: pd.Series, short_entries: pd.Series, size_series: pd.Series) -> vbt.Portfolio:
    # Ensure indices match
    index = price_series.index.union(long_entries.index).union(short_entries.index).union(size_series.index).sort_values()

    close = price_series.reindex(index).ffill()
    long_entries = long_entries.reindex(index, fill_value=False)
    short_entries = short_entries.reindex(index, fill_value=False)
    size = size_series.reindex(index).fillna(0)

    return vbt.Portfolio.from_signals(
        close=close,
        entries=long_entries,
        exits=short_entries,
        size=size,
        init_cash=0.0,
        direction="both",  # allow both long and short
        freq="1h"
    )



def analyze_wallet_balance(wallet_address:str, chain_name:str, days_lookback:int):
  daily_tokens_movement = take_daily_tokens_movement(wallet_address, chain_name, days_lookback)

  used_tokens = set(token for tokens in daily_tokens_movement.values() for token in tokens)

  starting_block = take_starting_block_height(chain_name, days_lookback)

  portfolios = {}
  for token_address in used_tokens:
    token_transfers = take_all_erc_token_transfers(wallet_address, chain_name, token_address, starting_block)
    backtesting_results = backtest_token_portfolio(token_address, wallet_address, token_transfers)
    portfolios[token_address] = backtesting_results
  
  return portfolios


portfolios = analyze_wallet_balance("0x7bfee91193d9df2ac0bfe90191d40f23c773c060", "eth-mainnet", 300)
print(portfolios)

{'0x624d822934e87d3534e435b83ff5c19769efd9f6': None, '0x0daa35a1735152edb928239c1712883dd8eb976f': None, '0xe4cf2d4eb9c01784798679f2fed4cf47cc59a3ec': None, '0xe19d1c837b8a1c83a56cd9165b2c0256d39653ad': None, '0x067e11ac5471c853aea205b3c1933a5f6367152f': None, '0x917cee801a67f933f2e6b33fc0cd1ed2d5909d88': <vectorbt.portfolio.base.Portfolio object at 0xff9d1e73f2c0>, '0xcde5d40f312b9bcf704babcdb6713d2547a277c4': None, '0xad1c5bc8882e87af14b16c51001bcedf253aedc1': None, '0x65c46c9fa2fb658bd25c1a973d25db2cd388d8ea': None, '0xf7cb66145c5fbc198cd4e43413b61786fb12df95': None, '0x5f98805a4e8be255a32880fdec7f6728c6568ba0': <vectorbt.portfolio.base.Portfolio object at 0xff9d1e73eb40>, '0x7a56e1c57c7475ccf742a1832b028f0456652f97': <vectorbt.portfolio.base.Portfolio object at 0xff9d1e73db80>, '0x72d120b9c47da21ed704e843b2c970ca743db175': None, '0x3def33ec7bbda557daf3995b2ecc5c87ad9d07e7': None, '0x9d6ec7a7b051b32205f74b140a0fa6f09d7f223e': None, '0x8668a15b7b023dc77b372a740fcb8939e15257cf': None,

In [30]:
for token_address, portfolio in portfolios.items():
  if not portfolio is None and portfolio.stats()["Total Trades"] > 0:
    if not portfolio.stats().get("Win Rate [%]", None) is None:
      if 0 < portfolio.stats()["Win Rate [%]"] < 50:
        print(portfolio.stats()["Win Rate [%]"])

12.5
14.285714285714285
10.0
28.57142857142857
42.857142857142854
33.33333333333333
33.33333333333333
14.285714285714285
40.0
33.33333333333333
28.57142857142857
33.33333333333333
10.526315789473683
45.45454545454545
25.0
